# Load libraries and Themes

In [1]:
suppressPackageStartupMessages({
    suppressWarnings({
        library(Seurat)
        library(ggplot2)
        library(tidyverse)
        library(SeuratDisk)
        library(DESeq2)
        library(sctransform)
        library(edgeR)
        library(EnhancedVolcano)
        library(Rsamtools)
        library(svglite)
        library(ComplexHeatmap)
        library(cowplot)
        library(svglite)
        library(viridis)
        library(pals)
        library(harmony)
        library(WGCNA)
        library(hdWGCNA)
        library(CellChat)
        library(reticulate)
        library(svglite)
        library(wordcloud)
        library(gridExtra)
        library(pals)
        library(patchwork)
        library(UpSetR)  
        library(ComplexUpset)
        library(sva)
        })})

In [ ]:
#Color Palettes

palette.CM <- c(
    "#67001f", # Deep red  
    "#b2182b", # Crimson  
    "#d6604d", # Coral red  
    "#f4a582", # Soft red  
    "#fddbc7"  # Pale red  
)

palette.CMcomb <- c(
    "#aad902", #BC 
    "#990902", #CM
    "#aba790", #CYC
    "#005f99", #ECart   
    "#05b1eb", #ECcap
    "#0303a3", #ECend
    "#003ae8", #EClym    
    "#ebe6c7", #FB
    "#077a01", #MP
    "#422c00", #PER
    "#FFD100", #SC
    "#ffab00", #SMC
    "#56B400"  #TC
)
   
palette.treatment <- c(
    "#C1C1C1", # CTRL
    "#AF0000", # ALDO
    "#006AF3"  # REC
)

palette.sex <- c(
    "#006AF3", 
    "#AF0000" ) #female

In [ ]:
umap_theme <- theme(
  axis.line=element_blank(),
  axis.text.x=element_blank(),
  axis.text.y=element_blank(),
  axis.ticks=element_blank(),
  axis.title.x=element_blank(),
  axis.title.y=element_blank(),
  panel.background=element_blank(),
  panel.border=element_blank(),
  panel.grid.major=element_blank(),
  panel.grid.minor=element_blank()
)

In [ ]:
setwd("/media/daten/dmeral/scseq_analysis/2024_LV_CTRL_ALDO_REC")

In [ ]:
celltype <- "CM"

# Subcluster analysis

In [ ]:
obj <- LoadH5Seurat("seurat_objects/obj_seu_merge_harmony_sgl_addmodule_rename_CMcomb_onlyprotcod_ccscore.h5seurat")
obj_full <- obj

In [ ]:
# Subset to major clusters
obj_sub <- subset(x = obj, subset = cell_type_CMcomb %in% c(celltype))

In [ ]:
# Get genes with non-zero counts
counts <- GetAssayData(obj_sub, layer = "counts")[,]
nonzero <- as.data.frame(rowSums(counts) > 0)
names(nonzero)[names(nonzero) == "rowSums(counts) > 0"] <- "nonzerofeature"
nonzero <- filter(nonzero, nonzerofeature == TRUE)
nonzero$names <- rownames(nonzero)
nonzero$nonzerofeature <- NULL
write.table(nonzero, paste0("nonzerocounts/nonzerocounts_", celltype, ".csv"), sep = ",", quote = FALSE,  row.names = FALSE, col.names = FALSE)

## CM subcluster

In [ ]:
protein_coding_genes <- unlist(read.csv("DEGs/nothreshold/protein_coding_gene_names_filtered.txt", header = TRUE, stringsAsFactors = FALSE))

# Ensure the protein-coding genes are present in the dataset
selected_genes <- protein_coding_genes[protein_coding_genes %in% rownames(obj_sub)]

# Normalize and scale the data for the selected genes
obj_sub <- ScaleData(obj_sub, features = selected_genes, verbose = FALSE)

# Run PCA using only the selected protein-coding genes
obj_sub <- RunPCA(obj_sub, features = selected_genes, npcs = 35, verbose = FALSE)

In [ ]:
# integrate "batch"
obj_sub$batch <- as.factor(obj_sub$batch)

obj.subcluster <- obj_sub %>% 
  RunHarmony(group.by.vars = c("batch"), theta = c(3), lambda = c(0.4), max_iter = 20, early_stop = FALSE, plot_convergence = FALSE, assay.use = "RNA", verbose = FALSE)

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 2.5, repr.plot.res = 300) 

dim1 <- DimPlot(obj_sub, reduction = "pca", group.by = "batch", dims = c(1, 2), cols = palette.batch) + ggtitle("PC batch before harmony") # Before
dim2 <- DimPlot(obj.subcluster, reduction = "harmony", group.by = "batch", dims = c(1, 2), cols = palette.batch) + ggtitle("PC batch after harmony") # After

ggsave(paste0("subcluster/", celltype, "/Plots/PC_batch_before_harmony.svg"), plot = dim1, units = "cm", dpi = 300, width = 15, height = 10)
ggsave(paste0("subcluster/", celltype, "/Plots/PC_batch_after_harmony.svg"), plot = dim2, units = "cm", dpi = 300, width = 15, height = 10)

In [ ]:
#Run UMAP
obj.subcluster <- obj.subcluster %>%
  RunUMAP(dims = 1:35,  spread = 2, min.dist = 0.3, reduction = "harmony", verbose = FALSE) %>%
  FindNeighbors(dims = 1:35, reduction = "harmony", verbose = FALSE) %>%
  FindClusters(resolution = 0.05)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 4, repr.plot.res = 300) 
# palette.10 <- DiscretePalette(10, palette = "stepped", shuffle = TRUE)

seurat_clusters <- DimPlot(obj.subcluster,  group.by = "seurat_clusters",  pt.size = 1, label = TRUE, shuffle = FALSE, label.size = 10, cols = palette.CM) + umap_theme & NoLegend()
treatment <- DimPlot(obj.subcluster, pt.size = 1, group.by = "treatment", shuffle = TRUE, cols = palette.treatment) + umap_theme & NoLegend()
sex <- DimPlot(obj.subcluster, pt.size = 1,  group.by = "sex", shuffle = TRUE, label.size = 10, , cols = palette.sex) + umap_theme
batch <- DimPlot(obj.subcluster, pt.size = 1,  group.by = "batch", shuffle = TRUE, label.size = 10, cols = palette.treatment) + umap_theme

seurat_clusters|treatment|sex|batch

In [ ]:
# Rename Idents and add cell_type_sub
cluster_annotations <- c("CM_0", "CM_1", "CM_2", "CM_3", "CM_4")
names(cluster_annotations) <- levels(obj.subclusterM)
obj.subcluster <- RenameIdents(obj.subcluster, cluster_annotations)
obj.subcluster$cell_type_sub <- Idents(obj.subcluster)

In [ ]:
# Plot and save plots
cell_type_sub <- DimPlot(obj.subcluster_CM, group.by = "cell_type_sub", pt.size = 1, label = TRUE, shuffle = F, label.size = 10, cols = palette.CM) + umap_theme & NoLegend()

ggsave(paste0("subcluster/", celltype, "/Plots/", celltype, "_cell_type_sub.svg"), plot = cell_type_sub, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("subcluster/", celltype, "/Plots/", celltype, "_seurat_clusters.svg"), plot = seurat_clusters, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("subcluster/", celltype, "/Plots/", celltype, "_treatment.svg"), plot = treatment, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("subcluster/", celltype, "/Plots/", celltype, "_sex.svg"), plot = sex, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("subcluster/", celltype, "/Plots/", celltype, "_batch.svg"), plot = batch, units = "cm", dpi = 300, width = 18, height = 15)

In [ ]:
treatment <- DimPlot(obj.subcluster, pt.size = 1, group.by = "treatment", shuffle = TRUE, cols = palette.treatment, split.by = "treatment") + umap_theme & NoLegend()
treatment

ggsave(paste0("subcluster/", celltype, "/Plots/", celltype, "_treatment_split.svg"), plot = treatment, units = "cm", dpi = 300, width = 18*3, height = 15)

In [ ]:
SaveH5Seurat(obj.subcluster, paste0("subcluster/", celltype, "/Subcluster_", celltype))

## FindAllMarkers

In [ ]:
all.markers <- FindAllMarkers(obj.subcluster, only.pos = FALSE, test.use = "wilcox")

write.csv(all.markers, file = paste0("subcluster/", celltype, "/all.markers_" , celltype, "_all.csv"))

## Cellnumbers per cluster and sample

In [ ]:
print("Cellnumbers per cluster and sample")
table <- table(obj.subcluster@meta.data$treatment, 
      obj.subcluster@meta.data$cell_type_sub)
table
write.csv(table, file = paste0("subcluster/", celltype, "/number_perCluster_", celltype, ".csv"))

print("Cellnumbers per sample and sample")
table <- table(obj.subcluster@meta.data$sample_id, 
      obj.subcluster@meta.data$cell_type_sub)
table
write.csv(table, file = paste0("subcluster/", celltype, "/number_persample_", celltype, ".csv"))

## DEG analysis

### WR sum

In [ ]:
# Perform Wilcoxon DEG analysis between CM_0 and CM_1
deg_CM0_vs_CM1 <- FindMarkers(
  obj.subcluster, 
  ident.1 = "CM_0", 
  ident.2 = "CM_1", 
  group.by = "cell_type_sub",
  test.use = "wilcox"
)

In [ ]:
# Save results to CSV
write.csv(deg_CM0_vs_CM1, file = paste0("subcluster/", celltype , "/DEGs_CM0_vs_CM1.csv"))

In [ ]:
# Load the list of protein-coding genes
protein_coding_genes <- unlist(read.csv("DEGs/nothreshold/protein_coding_gene_names_filtered.txt", header = TRUE, stringsAsFactors = FALSE))

# Load the DEG file
deg <- read.csv(paste0("subcluster/", celltype, "/DEGs_CM0_vs_CM1.csv"), header = TRUE, row.names = 1)

# Add filtering criteria
deg <- cbind(deg, Gene_names = rownames(deg))

# Filter for protein-coding genes, remove GM... and ...Rik, and non-mt genes
deg_filtered <- deg %>%
  filter(
    Gene_names %in% protein_coding_genes               # Keep only protein-coding genes
  )

# Remove the Gene_names column
deg_filtered <- deg_filtered[, !names(deg_filtered) %in% c("Gene_names")]

# Save the filtered file
write.csv(deg_filtered, file = paste0("subcluster/", celltype, "/DEGs_CM0_vs_CM1_filtered.csv"))

### pseudo-bulk PCA plot

In [ ]:
# Aggregate counts by `sample_id`
obj.subcluster$tsid <- paste0(obj.subcluster$treatment, "_", obj.subcluster$sample_id)

cts <- AggregateExpression(
  obj.subcluster,
  group.by = "tsid",  # Only group by combined column tsid
  assays = "RNA",
  slot = "counts",
  return.seurat = FALSE
)

cts <- cts$RNA  # Extract the counts matrix

# Filter counts for only protein-coding genes
cts_filtered <- cts[rownames(cts) %in% protein_coding_genes, ]

# Log-transform counts (avoid log(0) issues)
log_counts <- log1p(cts_filtered)

# Remove zero-variance genes
log_counts_filtered <- log_counts[apply(log_counts, 1, var) > 0, ]

# Ensure that `tsid` is correctly defined in the metadata
# Extract the `batch_info` corresponding to the 12 aggregated samples
batch_info <- obj.subcluster@meta.data$batch[match(colnames(log_counts_filtered), obj.subcluster@meta.data$tsid)]

# Add the batch info to the log_counts_filtered matrix as an additional row
log_counts_filtered_with_batch <- rbind(log_counts_filtered, batch = batch_info)

# Now, apply ComBat for batch effect correction using the log-transformed counts with batch information
adjusted_counts <- ComBat(
    dat = log_counts_filtered_with_batch[1:nrow(log_counts_filtered), ],  # Only include gene expression matrix
    batch = log_counts_filtered_with_batch["batch", ],  # Use the batch info row
    mod = NULL,                   # Model matrix 
    par.prior = TRUE,             # Use empirical Bayes method
    prior.plots = FALSE           # Don't plot the correction
)

# Perform PCA on the adjusted counts
pca_result <- prcomp(t(adjusted_counts), center = TRUE, scale. = TRUE)

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4, repr.plot.res = 300)

# Create a data frame for plotting
pca_df <- data.frame(
  PC1 = pca_result$x[, 1],  # First principal component
  PC2 = pca_result$x[, 2],  # Second principal component
  tsid = rownames(pca_result$x),  # Use rownames of pca_result$x for tsid
  treatment = gsub("_.*", "", rownames(pca_result$x))  # Extract treatment from tsid
)

# Ensure treatment is a factor with levels in the correct order
pca_df$treatment <- factor(pca_df$treatment, levels = c("CTRL", "ALDO", "REC"))

# Plot the PCA
PCA_plot <- ggplot(pca_df, aes(x = PC1, y = PC2, color = treatment, label = substr(tsid, nchar(tsid)-3, nchar(tsid)))) +
  geom_point(size = 4, alpha = 0.8) +
  geom_text(vjust = 2, hjust = 0.5, size = 3, color = "black") +  # Adjust vjust to place labels below
  theme_minimal() +
  labs(
    title = paste0("PCA of Pseudobulk Samples for ", celltype ," Post-ComBat Correction"),
    x = paste0("PC1 (", round(summary(pca_result)$importance[2, 1] * 100, 1), "% Variance)"),
    y = paste0("PC2 (", round(summary(pca_result)$importance[2, 2] * 100, 1), "% Variance)")
  ) +
  scale_color_manual(values = palette.treatment) +  # Correct color palette
  theme(
    legend.position = "right",
    text = element_text(size = 12),
    plot.title = element_text(size = 12),  # Decrease title size
    panel.grid.minor = element_blank(),    # Remove minor gridlines only
    panel.border = element_rect(color = "black", fill = NA, size = 0.5)  # Add black border around the plot
  )

PCA_plot
ggsave(paste0("subcluster/", celltype, "/Plots/PCA_plot_pseudobulk.svg"), plot = PCA_plot, units = "cm", dpi = 300, width = 15, height = 10)

# hdWGCNA

In [ ]:
# List with only protein-coding Genes

protein_coding_genes <- read.csv("DEGs/nothreshold/protein_coding_gene_names_filtered.txt", 
                                 header = FALSE, 
                                 stringsAsFactors = FALSE)[,1]

protein_coding_genes <- protein_coding_genes[-1]

In [ ]:
# Filter the gene expression matrix in the Seurat object
seurat_obj <- obj.subcluster
seurat_obj <- subset(seurat_obj, features = protein_coding_genes)  

# Now proceed with WGCNA setup
seurat_obj <- SetupForWGCNA(
  seurat_obj,
  gene_select = "fraction",  # You can still use fraction-based selection
  fraction = 0.05,          
  wgcna_name = celltype
)

In [ ]:
# construct metacells in each group (make sure that min_cells>k)
seurat_obj <- MetacellsByGroups(
  seurat_obj = seurat_obj,
  group.by = c("treatment", "cell_type_sub"),
  reduction = "harmony", 
  assay = "RNA",
  slot = "counts",
  k = 20,                          
  max_shared = 20,         
  ident.group = "cell_type_sub",      
  min_cells = 50
)

In [ ]:
# normalize metacell expression matrix:
seurat_obj <- NormalizeMetacells(seurat_obj)

seurat_obj <- SetDatExpr(
  seurat_obj,
  group_name = c("CM_0", "CM_1", "CM_2", "CM_3", "CM_4"),  # cluster
  group.by = "cell_type_sub",                                  # the metadata column containing the cell type info. This same column should have also been used in MetacellsByGroups
  assay = "RNA",                                               # using RNA assay
  slot = "data"                                                # using normalized data
)

# Test different soft powers:
seurat_obj <- TestSoftPowers(
  seurat_obj,
  networkType = "signed")

# plot the results:
plot_list <- PlotSoftPowers(seurat_obj)

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4, repr.plot.res = 300)

# assemble with patchwork
wrap_plots(plot_list, ncol = 2)

In [ ]:
# construct co-expression network:
seurat_obj <- ConstructNetwork(
  seurat_obj, soft_power = 7,
  setDatExpr = FALSE,
  tom_name = paste0("LV_", celltype), overwrite_tom = TRUE)

In [ ]:
# Generate palette
palette_mod <- DiscretePalette(8, palette = "stepped", shuffle = TRUE)

# Function to check if a color is grey (low saturation)
is_grey <- function(color) {
  hsv_vals <- rgb2hsv(col2rgb(color))
  return(hsv_vals["s", ] < 0.2)  # Low saturation indicates grey
}

# Filter out greys
palette_mod <- palette_mod[!sapply(palette_mod, is_grey)]

# If too many colors were removed, regenerate until enough remain
while (length(palette_mod) < 8) {
  new_colors <- DiscretePalette(8, palette = "stepped", shuffle = TRUE)
  palette_mod <- unique(c(palette_mod, new_colors[!sapply(new_colors, is_grey)]))
  palette_mod <- palette_mod[1:8]  # Ensure exactly 9 colors
}

# Print final colors
print(palette_mod)

In [ ]:
# reset the module colors
seurat_obj <- ResetModuleColors(seurat_obj, palette_mod)

In [ ]:
svg(paste0("subcluster/", celltype, "/hdWGCNA/Dendrogram.svg"), width = 15 / 2.54, height = 10 / 2.54)  # Convert cm to inches
PlotDendrogram(seurat_obj, main = paste0(celltype, " hdWGCNA Dendrogram"))
dev.off()

In [ ]:
TOM <- GetTOM(seurat_obj)
head(TOM)

In [ ]:
# need to run ScaleData first or else harmony throws an error:
seurat_obj <- ScaleData(seurat_obj, features = VariableFeatures(seurat_obj))

# compute all MEs in the full single-cell dataset
seurat_obj <- ModuleEigengenes(
 seurat_obj,
 group.by.vars = "cell_type_sub")

In [ ]:
# harmonized module eigengenes:
hMEs <- GetMEs(seurat_obj)

# module eigengenes:
MEs <- GetMEs(seurat_obj, harmonized = FALSE)

In [ ]:
# compute eigengene-based connectivity (kME):
seurat_obj <- ModuleConnectivity(
  seurat_obj,
  group.by = "cell_type_sub", group_name = c("CM_0", "CM_1", "CM_2", "CM_3", "CM_4"))

In [ ]:
# rename the modules
seurat_obj <- ResetModuleNames(
  seurat_obj,
  new_name = paste0(celltype ,"-M")
)

In [ ]:
saveRDS(seurat_obj, file = paste0("subcluster/", celltype, "/hdWGCNA/hdWGCNA_" , celltype, ".rds"))

In [ ]:
svg(paste0("subcluster/", celltype, "/hdWGCNA/KMEs.svg"), width = 15 / 2.54, height = 15 / 2.54)  # Convert cm to inches
PlotKMEs(seurat_obj, ncol = 3)
dev.off()

In [ ]:
# get the module assignment table:
modules <- GetModules(seurat_obj)

# show the first 6 columns:
head(modules[,1:8])
write.csv(modules, file = paste0("subcluster/", celltype, "/hdWGCNA/modules_df.csv"))

In [ ]:
# get hub genes
hub_df <- GetHubGenes(seurat_obj, n_hubs = 25)

head(hub_df)
write.csv(hub_df, file = paste0("subcluster/", celltype, "/hdWGCNA/hub_df.csv"))

In [ ]:
# make a featureplot of hMEs for each module
plot_list <- ModuleFeaturePlot(
  seurat_obj,
  features = "hMEs", # plot the hMEs
  order = TRUE # order so the points with highest hMEs are on top
)
# stitch together with patchwork
combined_plot <- wrap_plots(plot_list, ncol = 3)

ggsave(
  filename = paste0("subcluster/", celltype, "/hdWGCNA/ModuleFeaturePlot.svg"),
  plot = combined_plot,  # The patchwork object
  width = 15 / 2.54, 
  height = 15 / 2.54, 
  units = "in",  # Inches (ggsave doesn't use cm natively)
  dpi = 300
)

In [ ]:
# Run module UMAP
seurat_obj <- RunModuleUMAP(
  seurat_obj,
  n_hubs = 10, # number of hub genes to include for the UMAP embedding
  n_neighbors = 15, # neighbors parameter for UMAP
  min_dist = 0.1 # min distance between points in UMAP space
)

# get the hub gene UMAP table from the seurat object
umap_df <- GetModuleUMAP(seurat_obj)

In [ ]:
# All Genes
svg(paste0("subcluster/", celltype, "/hdWGCNA/ModuleUMAPPlot.svg"), width = 15 / 2.54, height = 15 / 2.54)  # Convert cm to inches

UMAPmodule <- ModuleUMAPPlot(
  seurat_obj,
  edge.alpha = 0.25,
  sample_edges = TRUE,
  edge_prop = 0.05, # proportion of edges to sample (20% here)
  label_hubs = 2 ,# how many hub genes to plot per module?
  keep_grey_edges = FALSE
)

dev.off()

#### DMEs

In [ ]:
group1 <- seurat_obj@meta.data %>% subset(treatment == "CTRL") %>% rownames
group2 <- seurat_obj@meta.data %>% subset(treatment == "ALDO") %>% rownames
group3 <- seurat_obj@meta.data %>% subset(treatment == "REC") %>% rownames

##### ALDO vs CTRL

In [ ]:
DMEs <- FindDMEs(
  seurat_obj,
  barcodes1 = group2,
  barcodes2 = group1,
  test.use = "wilcox",
  wgcna_name = celltype
)

DMEs

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 100)

p <- PlotDMEsLollipop(
  seurat_obj, 
  DMEs, 
  wgcna_name = celltype, 
  pvalue = "p_val_adj"
)

ggsave(paste0("subcluster/", celltype, "/hdWGCNA/PlotDMEsLollipop_ALDOvsCTRL.svg"), plot = p, units = "cm", dpi = 300, width = 10, height = 10)

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 100)

p <- PlotDMEsVolcano(
  seurat_obj,
  DMEs,
  wgcna_name = celltype
)

p

##### REC vs ALDO

In [ ]:
DMEs <- FindDMEs(
  seurat_obj,
  barcodes1 = group3,
  barcodes2 = group2,
  test.use = "wilcox",
  wgcna_name = celltype
)

DMEs

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 100)

p <- PlotDMEsLollipop(
  seurat_obj, 
  DMEs, 
  wgcna_name = celltype, 
  pvalue = "p_val_adj"
)

ggsave(paste0("subcluster/", celltype, "/hdWGCNA/PlotDMEsLollipop_RECvsALDO.svg"), plot = p, units = "cm", dpi = 300, width = 10, height = 10)

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 100)

p <- PlotDMEsVolcano(
  seurat_obj,
  DMEs,
  wgcna_name = celltype
)

p

# Transfer Subcluster Annotations to Original obj

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10, repr.plot.res = 100)

DimPlot(obj_full, group.by = "cell_type_CMcomb")|
DimPlot(obj.subcluster, group.by = "cell_type_sub")

In [ ]:
obj_sub <- obj.subcluster

In [ ]:
seurat_objects <- list(obj.subcluster) # This can be done with several obj containing subsets of full obj

In [ ]:
set_cell_type_sub <- function(seurat_objects) {
  for (obj in seurat_objects) {
    obj@meta.data$cell_type_sub <- Idents(obj)
  }
}

set_cell_type_sub(seurat_objects)

In [ ]:
# Check cluster annotations in obj_full

unique(obj_full@meta.data$cell_type_CMcomb)

In [ ]:
# Subset all cells except "celltype" in cell_type_CMcomb

object_wo_celltype <- obj_full[, obj_full@meta.data$cell_type_CMcomb != celltype]

In [ ]:
# Check successful removal of celltype in object_wo_celltype

unique(object_wo_celltype@meta.data$cell_type_CMcomb)

In [ ]:
# Create empty cell_type_sub_num column in obj_full

obj_full$cell_type_sub <- NA

In [ ]:
# Extract meta.data

original_metadata <- obj_full@meta.data
wo_subcluster_metadata <- object_wo_celltype@meta.data
celltype_metadata <- obj.subcluster@meta.data


In [ ]:
# Reset rownames to a column in all data frames

original_metadata$barcode <- rownames(original_metadata)
wo_subcluster_metadata$barcode <- rownames(wo_subcluster_metadata)
celltype_metadata$barcode <- rownames(celltype_metadata)

In [ ]:
# Subset to barcode and cell_type_sub columns

original_metadata <- original_metadata[c("barcode", "cell_type_sub")]
wo_subcluster_metadata <- wo_subcluster_metadata[c("barcode", "cell_type_CMcomb")]
celltype_metadata <- celltype_metadata[c("barcode", "cell_type_sub")]

In [ ]:
# Merge df by barcode

merged_df <- merge(original_metadata, wo_subcluster_metadata, by = "barcode", all.x = TRUE)
merged_df <- merge(merged_df, celltype_metadata, by = "barcode", all.x = TRUE)

In [ ]:
head(merged_df)

In [ ]:
# re-name col names

names(merged_df) <- c("barcode", "original", "wo_subcluster", celltype)

In [ ]:
# create new merged column containing either original annotations or sub cluster annotations for each barcode

merged_df$merged_column <- coalesce(merged_df$wo_subcluster, merged_df[[celltype]])

In [ ]:
# subset to key cols

cell_type_sub_df <- merged_df[c("barcode", "merged_column")]

In [ ]:
# re-order cell_type_sub_df to match order in obj_full 

cell_type_sub_df <- cell_type_sub_df[match(original_metadata$barcode, cell_type_sub_df$barcode), ]

In [ ]:
# subset to key col, unlist and add new annotations to obj_full@meta.data

cell_type_sub_df <- cell_type_sub_df[c("merged_column")]
cell_type_sub_df <- unlist(cell_type_sub_df, use.names = FALSE)
obj_full$cell_type_sub <- cell_type_sub_df

In [ ]:
SaveH5Seurat(obj_full, paste0("subcluster/", celltype, "/full_obj_with_Subcluster_", celltype, "_annotations"))

In [ ]:
DimPlot(obj_full, group.by = "cell_type_sub")

# CellChat with CM subcluster annotations

In [ ]:
# Subset obj to not contain CYC cluster and split by treatment
obj_cc <- subset(obj_full, subset = cell_type_CMcomb != "CYC")

Idents(obj_cc) <- obj_cc$cell_type_sub
split_LV <- SplitObject(obj_cc, split.by = "treatment")

In [ ]:
palette.CMcomb.sc <- c(
    "#aad902", #BC 
    "#005f99", #ECart   
    "#05b1eb", #ECcap
    "#0303a3", #ECend
    "#003ae8", #EClym    
    "#ebe6c7", #FB
    "#077a01", #MP
    "#422c00", #PER
    "#FFD100", #SC
    "#ffab00", #SMC
    "#56B400",  #TC
    "#67001f", # Deep red  
    "#b2182b", # Crimson  
    "#d6604d", # Coral red  
    "#f4a582", # Soft red  
    "#fddbc7"  # Pale red  
)

## CellChat LV CTRL

In [ ]:
# Prepare metadata
data.input <- GetAssayData(split_LV$CTRL, assay = "RNA", layer = "data")
labels <- Idents(split_LV$CTRL)
meta <- data.frame(group = labels, row.names = names(labels))

# Create the CellChat object with the filtered gene data
cc_obj <- createCellChat(object = data.input, meta = meta, group.by = "group")
cc_obj <- addMeta(cc_obj, meta = meta, meta.name = "Labels")
cc_obj <- setIdent(cc_obj, ident.use = "Labels")

# Check the cell type levels in the identifiers
levels(cc_obj@idents)

# Load the CellChat database for mouse
CellChatDB <- CellChatDB.mouse
showDatabaseCategory(CellChatDB)
dplyr::glimpse(CellChatDB$interaction)

# Use the mouse CellChat database
CellChatDB.use <- CellChatDB
cc_obj@DB <- CellChatDB.use

# Perform CellChat analysis steps
cc_obj <- subsetData(cc_obj)
cc_obj <- identifyOverExpressedGenes(cc_obj)
cc_obj <- identifyOverExpressedInteractions(cc_obj)

# Project data based on protein-protein interactions (PPI)
cc_obj <- projectData(cc_obj, PPI.mouse)

# Compute communication probabilities
cc_obj <- computeCommunProb(cc_obj, raw.use = FALSE)

# Filter communication based on minimum number of cells
cc_obj <- filterCommunication(cc_obj, min.cells = 10)

# Compute communication probabilities by pathway
cc_obj <- computeCommunProbPathway(cc_obj)

# Aggregate the communication network
cc_obj <- aggregateNet(cc_obj)

# Save the CellChat object for further analysis
cc_obj_CTRL <- cc_obj
cc_obj <- netAnalysis_computeCentrality(object = cc_obj, slot.name = "netP", thresh = 0.05)

# Save the CellChat object as an RDS file
cc_LV_CTRL <- cc_obj
saveRDS(cc_obj, file = paste0("subcluster/", celltype, "CellChat/CellChat_LV_CTRL"))

In [ ]:
groupSize <- as.numeric(table(cc_LV_CTRL@idents))

svg(file = paste0("subcluster/", celltype, "/CellChat/Plots/CTRL/LV_CTRL_Number.svg"))
gg1 <- netVisual_circle(cc_LV_CTRL@net$count, vertex.weight = groupSize, vertex.label.cex = 2, weight.scale = TRUE, label.edge= TRUE, title.name = "Number of interactions", color.use = palette.CMcomb.sc) 
dev.off()

svg(file = paste0("subcluster/", celltype, "/CellChat/Plots/CTRL/LV_CTRL_Weight.svg"))
gg2 <- netVisual_circle(cc_LV_CTRL@net$weight, vertex.weight = groupSize, vertex.label.cex = 2, weight.scale = TRUE, label.edge= TRUE, title.name = "Interaction weights/strength", color.use = palette.CMcomb.sc)
dev.off()

svg(file = paste0("subcluster/", celltype, "/CellChat/Plots/CTRL/LV_CTRL_byCluster.svg"), width = 24, height = 18)
mat <- cc_LV_CTRL@net$weight
par(mfrow = c(5,5), xpd = TRUE)
for (i in 1:nrow(mat)) {
  mat1 <- matrix(0, nrow = nrow(mat), ncol = ncol(mat), dimnames = dimnames(mat))
  mat1[i, ] <- mat[i, ]
  netVisual_circle(mat1, vertex.size.max = 5, vertex.weight = groupSize, vertex.label.cex = 1.5, arrow.size = 0, weight.scale = TRUE, label.edge= TRUE, edge.weight.max = max(mat), title.name = rownames(mat)[i], color.use = palette.CMcomb.sc)
}
dev.off()

## CellChat LV ALDO

In [ ]:
# Prepare metadata
data.input <- GetAssayData(split_LV$ALDO, assay = "RNA", layer = "data")
labels <- Idents(split_LV$ALDO)
meta <- data.frame(group = labels, row.names = names(labels))

# Create the CellChat object with the filtered gene data
cc_obj <- createCellChat(object = data.input, meta = meta, group.by = "group")
cc_obj <- addMeta(cc_obj, meta = meta, meta.name = "Labels")
cc_obj <- setIdent(cc_obj, ident.use = "Labels")

# Check the cell type levels in the identifiers
levels(cc_obj@idents)

# Load the CellChat database for mouse
CellChatDB <- CellChatDB.mouse
showDatabaseCategory(CellChatDB)
dplyr::glimpse(CellChatDB$interaction)

# Use the mouse CellChat database
CellChatDB.use <- CellChatDB
cc_obj@DB <- CellChatDB.use

# Perform CellChat analysis steps
cc_obj <- subsetData(cc_obj)
cc_obj <- identifyOverExpressedGenes(cc_obj)
cc_obj <- identifyOverExpressedInteractions(cc_obj)

# Project data based on protein-protein interactions (PPI)
cc_obj <- projectData(cc_obj, PPI.mouse)

# Compute communication probabilities
cc_obj <- computeCommunProb(cc_obj, raw.use = FALSE)

# Filter communication based on minimum number of cells
cc_obj <- filterCommunication(cc_obj, min.cells = 10)

# Compute communication probabilities by pathway
cc_obj <- computeCommunProbPathway(cc_obj)

# Aggregate the communication network
cc_obj <- aggregateNet(cc_obj)

# Save the CellChat object for further analysis
cc_obj_ALDO <- cc_obj
cc_obj <- netAnalysis_computeCentrality(object = cc_obj, slot.name = "netP", thresh = 0.05)

# Save the CellChat object as an RDS file
cc_LV_ALDO <- cc_obj
saveRDS(cc_obj, file = paste0("subcluster/", celltype, "/CellChat/CellChat_LV_ALDO"))

In [ ]:
groupSize <- as.numeric(table(cc_LV_ALDO@idents))

svg(file = paste0("subcluster/", celltype, "/CellChat/Plots/ALDO/LV_ALDO_Number.svg"))
gg1 <- netVisual_circle(cc_LV_ALDO@net$count, vertex.weight = groupSize, vertex.label.cex = 2, weight.scale = TRUE, label.edge= TRUE, title.name = "Number of interactions", color.use = palette.CMcomb.sc) 
dev.off()

svg(file = paste0("subcluster/", celltype, "/CellChat/Plots/ALDO/LV_ALDO_Weight.svg"))
gg2 <- netVisual_circle(cc_LV_ALDO@net$weight, vertex.weight = groupSize, vertex.label.cex = 2, weight.scale = TRUE, label.edge= TRUE, title.name = "Interaction weights/strength", color.use = palette.CMcomb.sc)
dev.off()

svg(file = paste0("subcluster/", celltype, "/CellChat/Plots/ALDO/LV_ALDO_byCluster.svg"), width = 24, height = 18)
mat <- cc_LV_ALDO@net$weight
par(mfrow = c(5,5), xpd = TRUE)
for (i in 1:nrow(mat)) {
  mat1 <- matrix(0, nrow = nrow(mat), ncol = ncol(mat), dimnames = dimnames(mat))
  mat1[i, ] <- mat[i, ]
  netVisual_circle(mat1, vertex.size.max = 5, vertex.weight = groupSize, vertex.label.cex = 1.5, arrow.size = 0, weight.scale = TRUE, label.edge= TRUE, edge.weight.max = max(mat), title.name = rownames(mat)[i], color.use = palette.CMcomb.sc)
}
dev.off()

## CellChat LV REC

In [ ]:
# Prepare metadata
data.input <- GetAssayData(split_LV$REC, assay = "RNA", layer = "data")
labels <- Idents(split_LV$REC)
meta <- data.frame(group = labels, row.names = names(labels))

# Create the CellChat object with the filtered gene data
cc_obj <- createCellChat(object = data.input, meta = meta, group.by = "group")
cc_obj <- addMeta(cc_obj, meta = meta, meta.name = "Labels")
cc_obj <- setIdent(cc_obj, ident.use = "Labels")

# Check the cell type levels in the identifiers
levels(cc_obj@idents)

# Load the CellChat database for mouse
CellChatDB <- CellChatDB.mouse
showDatabaseCategory(CellChatDB)
dplyr::glimpse(CellChatDB$interaction)

# Use the mouse CellChat database
CellChatDB.use <- CellChatDB
cc_obj@DB <- CellChatDB.use

# Perform CellChat analysis steps
cc_obj <- subsetData(cc_obj)
cc_obj <- identifyOverExpressedGenes(cc_obj)
cc_obj <- identifyOverExpressedInteractions(cc_obj)

# Project data based on protein-protein interactions (PPI)
cc_obj <- projectData(cc_obj, PPI.mouse)

# Compute communication probabilities
cc_obj <- computeCommunProb(cc_obj, raw.use = FALSE)

# Filter communication based on minimum number of cells
cc_obj <- filterCommunication(cc_obj, min.cells = 10)

# Compute communication probabilities by pathway
cc_obj <- computeCommunProbPathway(cc_obj)

# Aggregate the communication network
cc_obj <- aggregateNet(cc_obj)

# Save the CellChat object for further analysis
cc_obj_REC <- cc_obj
cc_obj <- netAnalysis_computeCentrality(object = cc_obj, slot.name = "netP", thresh = 0.05)

# Save the CellChat object as an RDS file
cc_LV_REC <- cc_obj
saveRDS(cc_obj, file = paste0("subcluster/", celltype, "/CellChat/CellChat_LV_REC"))

In [ ]:
groupSize <- as.numeric(table(cc_LV_REC@idents))

svg(file = paste0("subcluster/", celltype, "/CellChat/Plots/REC/LV_REC_Number.svg"))
gg1 <- netVisual_circle(cc_LV_REC@net$count, vertex.weight = groupSize, vertex.label.cex = 2, weight.scale = TRUE, label.edge= TRUE, title.name = "Number of interactions", color.use = palette.CMcomb.sc) 
dev.off()

svg(file = paste0("subcluster/", celltype, "/CellChat/Plots/REC/LV_REC_Weight.svg"))
gg2 <- netVisual_circle(cc_LV_REC@net$weight, vertex.weight = groupSize, vertex.label.cex = 2, weight.scale = TRUE, label.edge= TRUE, title.name = "Interaction weights/strength", color.use = palette.CMcomb.sc)
dev.off()

svg(file = paste0("subcluster/", celltype, "/CellChat/Plots/REC/LV_REC_byCluster.svg"), width = 24, height = 18)
mat <- cc_LV_REC@net$weight
par(mfrow = c(5,5), xpd = TRUE)
for (i in 1:nrow(mat)) {
  mat1 <- matrix(0, nrow = nrow(mat), ncol = ncol(mat), dimnames = dimnames(mat))
  mat1[i, ] <- mat[i, ]
  netVisual_circle(mat1, vertex.size.max = 5, vertex.weight = groupSize, vertex.label.cex = 1.5, arrow.size = 0, weight.scale = TRUE, label.edge= TRUE, edge.weight.max = max(mat), title.name = rownames(mat)[i], color.use = palette.CMcomb.sc)
}
dev.off()

## Compare the total number of interactions and interaction strength (thresh = 0.05)
To identify the interaction between which cell populations showing significant changes, CellChat compares the number of interactions and interaction strength among different cell populations.

In [ ]:
object.list <- list(CTRL = cc_LV_CTRL, ALDO = cc_LV_ALDO, REC = cc_LV_REC)
 
cellchat <- mergeCellChat(object.list, add.names = names(object.list))

# Merge the following slots: 'data.signaling','net', 'netP','meta', 'idents', 'var.features' , 'DB', and 'LR'.

# Check the merged object to ensure everything is correct
print(cellchat)

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 3, repr.plot.res = 300)

gg1 <- compareInteractions(cellchat, show.legend = FALSE, group = c(1 ,2, 3), color.use = palette.treatment, title.name = "Number of interactions", size.text = 20)
gg2 <- compareInteractions(cellchat, show.legend = FALSE, group = c(1 ,2, 3), measure = "weight", color.use = palette.treatment, title.name = "Strength of interactions", size.text = 20)
gg1 + gg2

ggsave(paste0("subcluster/", celltype, "/CellChat/Plots/NumberOfInteractions.svg"), plot = gg1, units = "cm", dpi = 300, width = 10, height = 10)
ggsave(paste0("subcluster/", celltype, "/CellChat/Plots/StrengthOfInteractions.svg"), plot = gg2, units = "cm", dpi = 300, width = 10, height = 10)

In [ ]:
# LV Diff Interaction Chord Diagrams

svg(file = paste0("subcluster/", celltype, "/CellChat/Plots/diffInteraction_ALDOvsCTRL.svg"), width = 14, height = 7)
par(mfrow = c(1, 2), xpd = TRUE)
PlotNetdiff_Num <- netVisual_diffInteraction(cellchat, comparison = c(1, 2), weight.scale = TRUE, vertex.label.cex = 2, arrow.size = 0.5, measure = "count.merged", color.use = palette.CMcomb.sc) # placeholder
PlotNetdiff_Wght <- netVisual_diffInteraction(cellchat, comparison = c(1, 2), weight.scale = TRUE, vertex.label.cex = 2, arrow.size = 0.5, measure = "weight", color.use = palette.CMcomb.sc)
dev.off()

svg(file = paste0("subcluster/", celltype, "/CellChat/Plots/diffInteraction_RECvsCTRL.svg"), width = 14, height = 7)
par(mfrow = c(1, 2), xpd = TRUE)
PlotNetdiff_Num <- netVisual_diffInteraction(cellchat, comparison = c(3, 1), weight.scale = TRUE, vertex.label.cex = 2, arrow.size = 0.5, color.use = palette.CMcomb.sc)
PlotNetdiff_Wght <- netVisual_diffInteraction(cellchat, comparison = c(3, 1), weight.scale = TRUE, vertex.label.cex = 2, arrow.size = 0.5, measure = "weight", color.use = palette.CMcomb.sc)
dev.off()

svg(file = paste0("subcluster/", celltype, "/CellChat/Plots/diffInteraction_RECvsALDO.svg"), width = 14, height = 7)
par(mfrow = c(1, 2), xpd = TRUE)
PlotNetdiff_Num <- netVisual_diffInteraction(cellchat, comparison = c(2, 3), weight.scale = TRUE, vertex.label.cex = 2, arrow.size = 0.5, color.use = palette.CMcomb)
PlotNetdiff_Wght <- netVisual_diffInteraction(cellchat, comparison = c(2, 3), weight.scale = TRUE, vertex.label.cex = 2, arrow.size = 0.5, measure = "weight", color.use = palette.CMcomb.sc)
dev.off()

In [ ]:
# Diff Interaction Heatmaps
options(repr.plot.width = 7, repr.plot.height = 6, repr.plot.res = 300)

ht1 <- netVisual_heatmap(cellchat, comparison = c(1, 2), width = 10, height = 28, color.use = palette.CMcomb.sc, color.heatmap = c("#0000c0", "#a00000"))
ht2 <- netVisual_heatmap(cellchat, comparison = c(1, 2), width = 10, height = 28, measure = "weight", color.use = palette.CMcomb.sc, color.heatmap = c("#0000c0", "#a00000"))
ht3 <- netVisual_heatmap(cellchat, comparison = c(2, 3), width = 10, height = 28, color.use = palette.CMcomb.sc, color.heatmap = c("#0000c0", "#a00000"))
ht4 <- netVisual_heatmap(cellchat, comparison = c(2, 3), width = 10, height = 28, measure = "weight", color.use = palette.CMcomb.sc, color.heatmap = c("#0000c0", "#a00000"))
ht5 <- netVisual_heatmap(cellchat, comparison = c(3, 1), width = 10, height = 28, color.use = palette.CMcomb.sc, color.heatmap = c("#0000c0", "#a00000"))
ht6 <- netVisual_heatmap(cellchat, comparison = c(3, 1), width = 10, height = 28, measure = "weight", color.use = palette.CMcomb.sc, color.heatmap = c("#0000c0", "#a00000"))

grob_ht1 = grid.grabExpr(draw(ht1))
grob_ht2 = grid.grabExpr(draw(ht2))
grob_ht3 = grid.grabExpr(draw(ht3))
grob_ht4 = grid.grabExpr(draw(ht4))
grob_ht5 = grid.grabExpr(draw(ht5))
grob_ht6 = grid.grabExpr(draw(ht6))

grid_ht <- plot_grid(grob_ht1, grob_ht2, grob_ht3, grob_ht4, grob_ht5, grob_ht6, ncol = 2)
grid_ht
ggsave(paste0("subcluster/", celltype, "/CellChat/Plots/netVisual_heatmap.svg"), plot = grid_ht, units = "cm", dpi = 300, width = 18, height = 24)

In [ ]:
# Signaling Role Scatter
options(repr.plot.width = 10, repr.plot.height = 5, repr.plot.res = 300)

# lA
svg(file = paste0("subcluster/", celltype, "/CellChat/Plots/signalingRoleScatter.svg"), width = 15, height = 4.5)
num.link <- sapply(object.list, function(x) {rowSums(x@net$count) + colSums(x@net$count)-diag(x@net$count)})
weight.MinMax <- c(min(num.link), max(num.link)) # control the dot size in the different datasets
gg <- list()
for (i in 1:length(object.list)) {
  gg[[i]] <- netAnalysis_signalingRole_scatter(object.list[[i]], title = names(object.list)[i], weight.MinMax = weight.MinMax, color.use = palette.CMcomb.sc) + scale_y_continuous(limits = c(0, 15)) + scale_x_continuous(limits = c(0, 15))
}
patchwork::wrap_plots(plots = gg)
dev.off()

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 10, repr.plot.res = 300)



gg1 <- netVisual_bubble(cellchat, sources.use = c(3, 6, 12, 13), targets.use = c(3, 6, 12, 13),  comparison = c(1, 2), max.dataset = 2, title.name = "Increased signaling in ALDO ", angle.x = 90, remove.isolate = TRUE, color.heatmap = "viridis", color.text = palette.treatment)

gg1 <- gg1 + scale_fill_gradientn(colors = brewer.pal(9, "OrRd")) +
      scale_color_gradientn(colors = brewer.pal(9, "OrRd"))

ggsave(paste0("subcluster/", celltype, "/CellChat/Plots/netVisual_bubble_CM_CTRL_ALDO.svg"), plot = gg1, units = "cm", dpi = 300, width = 15, height = 25)

gg1

In [2]:
sessionInfo()

R version 4.2.2 (2022-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 22.04.2 LTS

Matrix products: default
BLAS/LAPACK: /media/daten/dmeral/micromamba/envs/user_R/lib/libopenblasp-r0.3.26.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] grid      stats4    stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] sva_3.46.0                  BiocParallel_1.32.5        
 [3] genefilter_1.80.3           mgcv_1.9-1                 
 [5] nlme_3.1-164                ComplexUpset_1.3.3         
 [7] UpSetR_1.4.0                patchwork_1.3.0            
 [9] gridExtra_2.3           